In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
!wget https://github.com/alexeygrigorev/mlbookcamp-code/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing-model.h5

In [ ]:
!python -V

In [ ]:
tf.__version__

In [ ]:
model = keras.models.load_model('clothing-model.h5')

In [ ]:
!wget http://bit.ly/mlbookcamp-pants -O pants.jpg

In [ ]:
import numpy as np

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [ ]:
img = load_img('pants.jpg', target_size=(299, 299))

x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [ ]:
preds = model.predict(X)

In [ ]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

### Convert Keras to TF-Lite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('clothing-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

In [ ]:
!ls -lh

In [ ]:
import tensorflow.lite as tflite

In [ ]:
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [ ]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [ ]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

### Removing TF dependency

In [ ]:
from PIL import Image

In [ ]:
with Image.open('pants.jpg') as img:
    img = img.resize((299, 299), Image.NEAREST)

In [ ]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [ ]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [ ]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [ ]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

### Simpler way

In [1]:
# import tensorflow.lite as tflite
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [2]:
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [3]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [4]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [5]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [6]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -1.8682901,
 'hat': -4.7612457,
 'longsleeve': -2.3169823,
 'outwear': -1.0625706,
 'pants': 9.8871565,
 'shirt': -2.8124304,
 'shoes': -3.6662836,
 'shorts': 3.200361,
 'skirt': -2.6023388,
 't-shirt': -4.835045}